In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gpytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
print(torch.__version__)

1.12.1+cu113


In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import catboost as cb
import math
from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared, RBF, DotProduct, Matern, ConstantKernel, WhiteKernel, PairwiseKernel, CompoundKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV


CTRL + F10

In [ ]:
folder = "drive/MyDrive/AML_Pr_1/"
just_scoring = False
if just_scoring:
  preprocessed = folder + "preprocessed_data/"
else:
  preprocessed = folder + "ready_data/"

In [ ]:
x_train = pd.read_csv(preprocessed + 'X_train_preprocessed.csv',index_col=['id'])
x_test = pd.read_csv(preprocessed + 'X_test_preprocessed.csv',index_col=['id'])
y_train = pd.read_csv(preprocessed + 'y_train_preprocessed.csv',index_col=['id'])
y_test = pd.read_csv(preprocessed + 'y_test_preprocessed.csv',index_col=['id'])

In [ ]:
x_train_DL = pd.read_csv(preprocessed + 'X_train_preprocessed_DL.csv',index_col=['id'])
x_test_DL = pd.read_csv(preprocessed + 'X_test_preprocessed_DL.csv',index_col=['id'])
y_train_DL = pd.read_csv(preprocessed + 'y_train_preprocessed_DL.csv',index_col=['id'])
y_test_DL = pd.read_csv(preprocessed + 'y_test_preprocessed_DL.csv',index_col=['id'])

In [ ]:
TRAIN_RATIO = 0.8

def performance(y_pred, y_true = y_test):
  varb = y_true.empty
  if not y_true.empty:
    return r2_score(y_true,y_pred)

def train_len(data = x_train):
  return math.floor(TRAIN_RATIO*len(data))

def validation_len(data = x_train):
    return len(data) - train_len(data)

def train(data = x_train):
    if isinstance(data, np.ndarray):
        return data[:train_len(data)].copy()
    return data.iloc[:train_len(data)].copy()

def validation(data = x_train):
    if isinstance(data, np.ndarray):
        return data[-validation_len(data):].copy()
    return data.iloc[-validation_len(data):].copy()

In [ ]:
x_train

,x2,x15,x21,x23,x27,x29,x40,x69,x77,x87,...,x773,x774,x777,x788,x796,x801,x817,x819,x823,x824
id,,,,,,,,,,,,,,,,,,,,,
0.0,-0.226678,-0.337851,-1.155732,0.599105,0.158027,-0.632436,0.495668,-0.494824,0.371962,0.436958,...,-0.311507,-0.265153,0.271874,-0.981772,-0.650906,-0.397318,0.871044,-0.181158,-0.412912,0.190751
1.0,1.587980,2.358662,1.345399,-0.121116,1.093737,0.658817,-1.435164,1.502663,0.376827,1.523190,...,1.711289,2.112223,0.399513,0.655927,0.497726,0.083890,-0.022647,1.789948,1.418251,-0.212470
2.0,-0.625908,-0.869980,-0.614856,0.005391,-0.406470,0.285827,0.356319,-0.368856,0.394291,0.298336,...,0.020761,-0.356562,0.087286,0.180971,-0.373038,0.639238,0.284144,-0.769021,0.360299,-0.437600
3.0,-2.929110,-3.152708,-0.441039,-1.832078,-3.384087,-0.505167,3.921911,-3.951556,-3.029386,-3.360842,...,-3.387495,-2.790336,-1.450183,-2.786642,-0.650906,-1.557200,-1.995884,-3.622290,-3.336300,-2.790125
4.0,-0.013582,-0.420911,1.085127,-0.267716,-0.174137,2.337714,-0.073531,0.093897,-0.829828,-0.111829,...,1.772192,-0.172693,-0.504449,-1.002283,1.632004,-0.648963,-0.549555,-0.002139,0.263169,0.016198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,0.859382,0.678270,1.629350,0.994234,1.528276,1.362006,-1.770572,1.835961,0.455996,0.490772,...,4.135244,0.891160,0.657093,1.857465,-0.604746,0.979299,-0.362689,1.224911,2.146213,1.269247
1208.0,0.065727,0.327383,0.030918,-0.794895,0.463941,1.049338,-0.764547,0.773116,-0.918908,0.768645,...,0.644394,0.124948,-0.323168,-0.111129,0.286177,-0.389131,-0.871130,0.595737,0.715023,-0.120347
1209.0,0.385426,0.385349,2.354932,0.848814,-0.163240,-0.305340,-0.259389,0.253541,0.728109,-0.076369,...,1.545145,0.025123,0.070586,-0.560445,0.342699,0.887566,0.335091,0.024908,0.955451,0.104321


In [ ]:
%%time
kernels = [ConstantKernel*Matern(nu = nu_hat) for nu_hat in [0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 20, 30]] + [RBF(length_scale = length_scale_hat) for length_scale_hat in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
gs_gpr = GridSearchCV(estimator=GaussianProcessRegressor(normalize_y=True, random_state=0),
                        param_grid={
                            #'normalize_y': [True, False]
                            #'kernel_nu' = [1, 30],
                            'kernel': kernels
                        },scoring='r2', n_jobs=-1)
gs_gpr.fit(np.array(x_train), np.ravel(y_train))


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
AttributeError: type object 'ConstantKernel' has no attribute 'log'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/_gpr.py", line 257, in fit
    if self.optimizer is not None and self.kernel_.n_dims > 0:
  File "/usr/loc

CPU times: user 7.31 s, sys: 2.09 s, total: 9.4 s
Wall time: 1min 29s


TypeError: ignored

TypeError: ignored

In [ ]:
gs_gpr.best_params_

{'kernel': RBF(length_scale=2)}

In [ ]:
y_pred_gpr = gs_gpr.predict(np.array(x_test))
print(performance(y_pred_gpr))

#0.4579207525257344 Matteo data preprocessing {'kernel': RBF(length_scale=5)}
#0.4814665809780151 k=100 {'kernel': RBF(length_scale=1)}
#0.46162433256435165 k=60 {'kernel': RBF(length_scale=4)}
#0.4886350803538446 k=80 {'kernel': RBF(length_scale=3)}

None


In [ ]:
cdsfs

NameError: ignored

In [ ]:
from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared, RBF, DotProduct, Matern, ConstantKernel, WhiteKernel, PairwiseKernel, CompoundKernel
from sklearn.gaussian_process import GaussianProcessRegressor
kernel = RBF() #* RBF(15) #RBF() # ConstantKernel()*Matern(nu=10)
gpc = GaussianProcessRegressor(normalize_y=True, kernel=kernel,random_state=0)
gpc.fit(np.array(x_train), np.ravel(y_train))
y_pred_gpr = gpc.predict(np.array(x_test))
y_pred_gpr = np.ravel(y_pred_gpr)
print(performance(y_pred_gpr))
#0.47950308120239216; 0.4814665809780151 k=100
#0.4620910737351668 k=120
#0.48863507787237304 k=80 length=5
#0.49517123211806957 k=80 nu=10

In [ ]:
for nu_hat in [1/100, 1/8, 1/4, 1/2, 1, 1.5, 2, 4, 8, 9, 10, 11, 12, 13, 14, 15, 20, 30]:
  kernel = ConstantKernel()*Matern(nu = nu_hat)

  # gs_gpr = GridSearchCV(estimator=GaussianProcessRegressor(kernel = ConstantKernel()*Matern(nu = 30)),
  #                        param_grid={
  #                            'normalize_y' = [True, False]
  #                            #'kernel_nu' = [1, 30],
  #                        },scoring='r2', n_jobs=-1)

  gpc = GaussianProcessRegressor(normalize_y=True, kernel=kernel,random_state=0)
  gpc.fit(np.array(train(x_train)), np.log(np.ravel(train(y_train))))
  y_pred_gpr = gpc.predict(np.array(validation(x_train)))
  y_pred_gpr = np.exp(np.ravel(y_pred_gpr))
  print(nu_hat, performance(y_pred_gpr, validation(np.ravel(y_train))))
#k= 80
#0.01 0.3104649036382263
# 0.125 0.45206076666715245
# 0.25 0.470372339077446
# 0.5 0.482032135663782
# 1 0.491988271096772
# 1.5 0.4944700646976439
# 2 0.5030820768894178
# 4 0.4995621685610028
# 8 0.48811495493903634
# 9 0.49417310791916036
# 10 0.49248617985044263
# 11 0.49359995468741336
# 12 0.4854556576809942
# 13 0.48497192618603446
# 14 0.48453772131275386
# 15 0.48415031655045115
# 20 0.4826707132925864
# 30 0.48098952731388855

In [ ]:
for nu_hat in [1, 1.5, 2, 4, 8, 9, 10, 11, 12, 13, 14, 15, 20, 30]:
  kernel = ConstantKernel()*Matern(nu = 3)

  # gs_gpr = GridSearchCV(estimator=GaussianProcessRegressor(kernel = ConstantKernel()*Matern(nu = 30)),
  #                        param_grid={
  #                            'normalize_y' = [True, False]
  #                            #'kernel_nu' = [1, 30],
  #                        },scoring='r2', n_jobs=-1)

  gpc = GaussianProcessRegressor(normalize_y=True, kernel=kernel,random_state=0)
  gpc.fit(np.array(x_train), np.log(np.ravel(y_train)))
  y_pred_gpr = gpc.predict(np.array(x_test))
  y_pred_gpr = np.exp(np.ravel(y_pred_gpr))
  print(nu_hat, performance(y_pred_gpr))
#0.4879798615863574 k=80
#0.4895 k=80 ConstantKernel()*Matern(nu = 10)
#0.4852071646091425 k=100 ConstantKernel()*Matern(nu = 3)

In [ ]:
rfr = ExtraTreesRegressor(n_jobs=-1, max_depth=None, n_estimators=341, random_state=0, min_samples_split=3,
                          max_features=None)

rfr.fit(x_train, np.ravel(y_train))

y_pred_extra_trees = rfr.predict(x_test)

y_fake_extra_trees = rfr.predict(x_train)

print(performance(y_pred_extra_trees))
#0.5356171300373443
#0.35532062406231
#0.3761936314067723
# 0.3897876247411667 k feature = 80
# 0.37449264029700235 k feature = 100
# 0.411107572271865 added_booleans k=100
#0.4251722556943349
#0.4243527813999528 k=120
#0.4325048275124873 k=100 n_estimators = 341
#0.42743202491834176 k=120
#k = 80 0.42126352746679385

None


In [1]:
%%time
rfr = ExtraTreesRegressor(n_jobs=-1, max_depth=None, random_state=0,
                          max_features=None,)

param_grid = {
    'n_estimators' : [250,300,341,350,400,500],
    #'max_depth': [None,10,50,100],
    'max_leaf_nodes':[None,10,50,100,200],
    'min_sample_split': [2, 3]
}

gcv = GridSearchCV(rfr,param_grid,cv=10,n_jobs=-1).fit(x_train,y_train.values.ravel())

#rfr.fit(x_train, np.ravel(y_train))

y_pred_extra_trees = gcv.predict(x_test)
print(gcv.best_params_)
print(performance(y_pred_extra_trees))
#Wall time: 1h 14min 35s

NameError: ignored

In [198]:
%%time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
level0 = list()
level0.append(('GP', gs_gpr))
level0.append(('extra_tree', gcv))
level1 = LinearRegression()
stacking_reg_4 = StackingRegressor(estimators=level0, final_estimator=level1, cv=10)
stacking_reg_4.fit(np.array(x_train), np.ravel(y_train))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
AttributeError: type object 'ConstantKernel' has no attribute 'log'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/_gpr.py", line 257, in fit
    if self.optimizer is not None and self.kernel_.n_dims > 0:
  File "/usr/loc

CPU times: user 44.8 s, sys: 16.8 s, total: 1min 1s
Wall time: 9min 39s


TypeError: ignored

TypeError: ignored

In [199]:
y_stacking_3 = stacking_reg_4.predict(x_test)
print(performance(y_stacking_3))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


None


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [200]:
output_df4 = pd.DataFrame(y_stacking_3, columns = ['y'])
output_df4.to_csv(preprocessed + 'output4.csv', index_label="id")
output_df4.to_csv(preprocessed + 'output4.txt', index_label="id")

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    n_estimators=80, random_state = 0
)
gbr.fit(np.array(x_train), np.ravel(y_train))
y_pred_gbr = gbr.predict(np.array(x_test))

y_fake_gbr = gbr.predict(np.array(x_train))

print(performance(y_pred_gbr))

#0.4109502743903839
#0.4366241930586553 k=120
#0.3964197552066536 k=80

In [ ]:
# rfr = ExtraTreesRegressor(n_jobs=-1, max_depth=None, n_estimators=500, random_state=0, min_samples_split=3,
#                           max_features=None)

# rfr.fit(x_train_DL, np.ravel(y_train_DL))

# y_pred_extra_trees = rfr.predict(x_test_DL)

# y_fake_extra_trees = rfr.predict(x_train)

# print(performance(y_pred_extra_trees, y_test_DL))

In [ ]:
x_train

In [ ]:
# %%time
# bootstrap_rfr = ExtraTreesRegressor(n_jobs=-1, max_depth=None, n_estimators=1000, random_state=0, min_samples_split=2,
#                           max_features=None, bootstrap = True)

# bootstrap_rfr.fit(x_train_DL, np.ravel(y_train_DL))

# y_pred_bootstrap_extra_trees = bootstrap_rfr.predict(x_test_DL)

# print(performance(y_pred_bootstrap_extra_trees, y_test_DL))

In [ ]:
%%time
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV

grid_svr = GridSearchCV(

estimator=svm.SVR(kernel='rbf', tol=1e-6, cache_size=1000),
        param_grid={
            'C': [20, 23, 25, 28, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 50, 80, 100], #[1/10, 1, 10, 25, 50, 100],
            'epsilon': [0.0001, 0.0003, 0.007, 0.0109, 0.019, 0.05, 0.06, 0.07, 0.08, 0.1, 0.15, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.35, 0.38, 0.4, 0.45, 0.47, 0.48, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 1, 1.1, 1.2, 1.3, 1.5, 2], #0.0001, 0.0003, 0.007, 0.0109, 0.019,0.05, 0.06, 0.07, 0.08, 0.1, 0.15, 0.2,
        },scoring='r2', n_jobs=-1)
grid_svr.fit(x_train, np.ravel(y_train))

#time: 8'11''

In [ ]:
print("best parameters are: ", grid_svr.best_params_)
# best parameters are:  {'C': 25, 'epsilon': 0.05, 'kernel': 'rbf'}
# {'C': 33, 'epsilon': 0.24}
# {'C': 35, 'epsilon': 0.8} k=80
# {'C': 35, 'epsilon': 1, 'gamma': 'scale'}

In [ ]:
y_pred_svr = grid_svr.predict(x_test)

y_fake_svr = grid_svr.predict(x_train)

print(performance(y_pred_svr))
#0.5356171300373443
#0.35532062406231
#0.08237266424460488
# 0.05912369533471484 k feature = 80
# 0.05282851077633721 k feature = 100
#0.45253244196980247
#0.4441209923670233 k=120
# 0.4548020958489001 k=100, C=33, epsilon=0.24
# 0.46056107018821146 k=80, c=35, epsilon=0.8
# 0.46103017650795386 k=80, C=35, epsilon=1, gamma=scale
# 0.4609875817093887 k=80 C=35, epsilon=1.2

In [ ]:
# regr = svm.SVR(kernel='rbf')
# regr.fit(x_train, np.ravel(y_train))
# test = regr.predict(x_test)

# print(performance(test))
# #0.09478917148751076
# # 0.008166515321314738 k = 80
# # 0.04154779942679787 k = 100

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr_random_forest = RandomForestRegressor(n_estimators=800, max_depth=10, random_state=0, n_jobs=-1) #max_depth
regr_random_forest.fit(x_train, np.ravel(y_train))
y_pred_random_for = regr_random_forest.predict(x_test)


y_fake_random_for = regr_random_forest.predict(x_train)

print(performance(y_pred_random_for))
#0.33815349663847327
#0.3573393167169928 k = 80
#0.3566131611070854 k = 100
#0.37512258402414267 k = 100
#0.37415137396623843
#0.3791667954394128 k = 120
#0.38263232701377947 k=100, n_estimators=800
#0.3815054780797936 k=120
#0.3737629780568452 k=80

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# regr_random_forest_DL = RandomForestRegressor(n_estimators=800, max_depth=10, random_state=0, n_jobs=-1)
# regr_random_forest_DL.fit(x_train_DL, np.ravel(y_train_DL))
# y_pred_random_for_DL = regr_random_forest.predict(x_test_DL)

# print(performance(y_pred_random_for_DL))

In [ ]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())

In [ ]:
%%time
model_cat = cb.CatBoostRegressor(random_seed=0)
grid = {'iterations': [150, 200, 250, 300],
        'learning_rate': [0.03, 0.1, 0.5],
        'depth': [1, 2, 3, 4, 5],
        'l2_leaf_reg': [0.05, 0.1, 0.25, 0.5, 1]}
model_cat.grid_search(grid, x_train, np.ravel(y_train),cv=10)
#Cross validation has to be done on validation set

In [ ]:
y_pred_catboost = model_cat.predict(x_test)

y_fake_catboost = model_cat.predict(x_train)

print(performance(y_pred_catboost))
#0.5365095729551678
#-0.22010201623213943
#0.3533494052043632
#0.32539542596473103 k = 80
#0.35410761873500995 k = 100
# 0.4067293468479508
# 0.42982089637247567 k = 120

In [ ]:
ensemble_pred  = (y_pred_extra_trees + y_pred_gpr)/2

print(performance(ensemble_pred))
#0.5465199080465408
#-0.5832279612528048
#0.378174825669665 last try
#0.3772323232079067 k = 80
#0.3822810642456935 k = 100
#0.40440458271179747 k = 100
# 0.42537767898333667; 0.4764978789716765
# 0.4737872721460522 k = 120
# 0.478541306383385 k=100
# 0.4747254051145726 k=120
# 0.4888026181212698 k=100 reduced

In [ ]:
if just_scoring:
  extra_trees_perf = performance(y_pred_extra_trees)
  random_for_perf = performance(y_pred_random_for)
  catboost_perf = performance(y_pred_catboost)
  svr_perf = performance(y_pred_svr)
  gpr_perf = performance(y_pred_gpr)
  gbr_perf = performance(y_pred_gbr)
  
  tot_perf = extra_trees_perf+gpr_perf

# extra_trees_perf = 0.4325048275124873
# random_for_perf = 0.38263232701377947
# catboost_perf = 0.4067293468479508

# svr_perf = 0.45253244196980247
# gpr_perf = 0.4814665809780151

# gbr_perf = 0.4109502743903839 #it's the same as catboost i believe so no need to include it

# k=100
# if k==120:
#   extra_trees_perf = 0.42743202491834176
#   random_for_perf = 0.3815054780797936
#   catboost_perf = 0.42982089637247567
#   svr_perf = 0.4441209923670233
#   gpr_perf = 0.4620910737351668
#   gbr_perf = 0.4366241930586553


# extra_trees_perf = -1/(extra_trees_perf-1)
# random_for_perf = -1/(random_for_perf-1)
# catboost_perf = -1/(catboost_perf-1)
# svr_perf = -1/(svr_perf-1)
# gpr_perf = -1/(gpr_perf-1)



In [ ]:
ensemble_pred_weighted  = (y_pred_extra_trees * extra_trees_perf +
                           y_pred_gpr * gpr_perf)/tot_perf
print(performance(ensemble_pred_weighted))
#0.3788420790651126 last try
#0.37787499102158084 k = 80
#0.3825656170855435 k = 100
#0.4048195117461517 k = 100
#0.42565518535609936; 0.4796283185620178
# 0.4759091874013891 k = 120
#0.48308720911116276 k=100 ??
#0.4763216381427766 k=120; 0.4778406755990815
#0.49002275929601835 k=100 reduced

In [ ]:
y_ensemblement = np.array([y_pred_extra_trees, y_pred_gpr, y_pred_gbr, y_pred_catboost, y_pred_svr, y_pred_random_for]).T
y_test.shape

In [ ]:
# from sklearn.feature_selection import f_regression
# f_values, p_values = f_regression(X = y_ensemblement, y = y_test)
# p_values

# y_fake_extra_trees = rfr.predict(x_train)
# y_fake_gpr = gpc.predict(x_train)
# y_fake_gbr = mod.predict(x_train)
# y_fake_catboost = model.predict(x_train)
# y_fake_random_for = regr_random_forest.predict(x_train)
#y_fake_svr = grid_svr.predict(x_train)

y_fake_ensemblement = np.array([y_fake_extra_trees, y_fake_gbr, y_fake_catboost, y_fake_random_for, y_fake_svr, y_fake_gpr]).T # #y_fake_svr
# y_fake_gpr is bad (makes no sense)
from sklearn.feature_selection import f_regression
f_values, p_values = f_regression(X = y_fake_ensemblement, y = np.ravel(y_train))
p_values

In [ ]:
from pandas.core.dtypes.cast import ensure_int64
from sklearn.linear_model import Ridge

if just_scoring:
  ens_lr = Ridge(fit_intercept=True, random_state = 0)
  ens_lr.fit(X = y_ensemblement, y = y_test)
  print(performance(ens_lr.predict(y_ensemblement)))
  # 0.5167025280991099
else:
  best_y = ens_lr.predict(y_ensemblement)




In [ ]:
output_df = pd.DataFrame(y_pred_gpr, columns = ['y'])
output_df

,y
0,58.596855
1,75.811544
2,67.985720
3,77.580724
4,73.865564
...,...
771,64.323890
772,70.320099
773,73.334406
774,71.174983


In [ ]:
output_df.to_csv(preprocessed + 'output.csv', index_label="id")
output_df.to_csv(preprocessed + 'output.txt', index_label="id")

In [ ]:
x_train

,x23,x27,x107,x113,x115,x133,x146,x159,x172,x194,...,x720,x721,x734,x745,x748,x766,x769,x777,x801,x817
id,,,,,,,,,,,,,,,,,,,,,
0.0,0.599105,0.158027,-0.332556,0.103688,1.225304,-0.227474,1.099541,1.038731,1.266459,0.133972,...,-0.373422,0.708957,0.718722,0.086823,1.115557,0.229200,0.314981,0.271874,-0.397318,0.871044
1.0,-0.121116,1.093737,-0.192778,1.088172,-0.494350,0.938117,1.104725,-0.840436,0.927798,0.076621,...,2.410055,0.170229,0.511155,0.849399,0.301195,0.214749,-1.231591,0.399513,0.083890,-0.022647
2.0,0.005391,-0.406470,0.695954,-0.898019,-0.455348,-0.406773,0.174580,-0.307028,0.321067,0.423860,...,-0.238233,-0.082025,1.064913,-0.066236,-0.054372,0.374432,0.320744,0.087286,0.639238,0.284144
3.0,-1.832078,-3.384087,-1.720375,-2.129512,1.166829,-2.985139,0.096687,0.774922,-1.939021,-2.378778,...,-0.284422,-1.508955,-1.828042,-1.490474,-2.081135,-2.041856,-1.704455,-1.450183,-1.557200,-1.995884
4.0,-0.267716,-0.174137,-0.079229,-0.018856,0.217443,0.303065,0.050915,0.033793,0.053500,0.290893,...,0.701988,0.318458,-0.987125,-0.030956,-0.331437,-0.162211,-0.679684,-0.504449,-0.648963,-0.549555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,0.994234,1.528276,0.446310,0.768486,-0.482549,0.741673,0.345159,-0.597957,1.629033,0.982215,...,-0.381401,0.491163,0.220182,-0.356397,0.594469,0.462098,-0.255938,0.657093,0.979299,-0.362689
1208.0,-0.794895,0.463941,-0.069020,-0.477726,-0.143477,-0.144869,-0.840329,0.033793,0.337580,-0.076261,...,1.343011,-0.356854,-0.711649,-0.409721,0.043962,-0.887702,0.153036,-0.323168,-0.389131,-0.871130
1209.0,0.848814,-0.163240,1.092785,0.245147,-0.413303,0.449169,0.705372,-0.587616,1.155228,0.481710,...,-0.210616,-0.173103,0.989269,1.353702,0.416287,1.781606,0.513185,0.070586,0.887566,0.335091
